In [3]:
import sqlite3
import pandas as pd
import numpy as np
import global_imports
import re
import json

# Not necessary in a py file as jupyter does not set the path correctly
import os
os.chdir('hard_to_choose/')

In [4]:
def import_database_from_sql(sql_query,db_path):
    connector = sqlite3.connect(db_path)
    db = pd.read_sql(sql_query, connector)
    return db

game_piece_type_query = 'SELECT * FROM GamePieceTypes'
game_piece_type = import_database_from_sql(game_piece_type_query, global_imports.main_db_path)

main_db_query = """SELECT GamePieces.releaseKey, GamePieces.gamePieceTypeId, GamePieces.value
    FROM GameLinks
	JOIN GamePieces ON GameLinks.releaseKey = GamePieces.releaseKey"""
main_db = import_database_from_sql(main_db_query, global_imports.main_db_path)

In [5]:
#Add any metadata needed to this list. Valid values are type names
# from GamePieceTypes
column_names = ['title', 'meta']

def split_metadata_into_columns(column_list, db):
    """Splits metadata of the main_db into different rows. Arguments are a list of column_names
    and the dataframe"""

# Creates columns for each item in column_list
    for val in column_list:
        type_id = int(game_piece_type.loc[game_piece_type['type'] == val, 'id'])
        db[val] = db.loc[db['gamePieceTypeId'] == type_id, 'value']

# Drop rows that are null in all of the generated columns. This will remove every row that
# is not in column_list
    db = db.dropna(subset=column_list, how='all')

# Drop every other row we do not need anymore
    db = db.drop(['value','gamePieceTypeId'], axis=1)

# Group by releaseKey since the data is in its own rows. Fill row with NAN if every row is NAN
    db = db.groupby('releaseKey', as_index=True).agg(lambda x: np.nan if x.isnull().all()
    else x.dropna()).reset_index()
    db = db.set_index(np.arange(1, len(db) + 1))
    return db


main_db = split_metadata_into_columns(column_names, main_db)

In [6]:
def extract_date(db):
    """Extract Release Date from metadata and convert it into MM YYYY format"""

    date_pattern = 'releaseDate\":(\d{9,10})'
    def format_date(x):
        date = re.search(date_pattern, x)
        if date:
            val = pd.to_datetime(date.group(1), unit='s')
            val = val.strftime('%Y %b')
            return val
        else:
            return 'No Date'

    db['date'] = db['meta'].apply(format_date)
    db = db.drop('meta', axis=1)
    return db

main_db = extract_date(main_db)

In [7]:
def create_platform(db):
    with open(global_imports.platforms_json) as platform_file:
        platform = json.load(platform_file)

    # Create a regex pattern of all platforms to match and then return the actual platform name
    platform_keys = list(platform.keys())
    platform_pattern = re.compile(r"(\b{}\b)".format("|".join(platform_keys)))

    def platform_extract(x):
        m = platform_pattern.match(x)
        if m:
            return platform[m.group(1)]

    db['platform'] = db['releaseKey'].apply(platform_extract)
    return db

main_db = create_platform(main_db)

In [19]:
def remove_hidden_games(db):
    hidden_db_query = 'SELECT releaseKey, isDlc, isVisibleInLibrary FROM ReleaseProperties'
    hidden_db = import_database_from_sql(hidden_db_query, global_imports.main_db_path)

# Merge is on left because every releaseKey in main_db is present in hidden_db
    db = db.merge(hidden_db, how='left', on='releaseKey')

# Delete everything marked as DLC or not visible in library
    db = db.drop(db[(db['isVisibleInLibrary'] == 0) | (db['isDlc'] == 1)].index)

    return db

main_db = remove_hidden_games(main_db)

,releaseKey,title,date,platform,isDlc,isVisibleInLibrary
0,battlenet_1146311730,"{""title"":""Destiny 2""}",2017 Sep,Battle.net,0,1
1,battlenet_1214607983,"{""title"":""Heroes of the Storm®""}",2015 Jun,Battle.net,0,1
2,battlenet_1465140039,"{""title"":""Hearthstone®""}",2014 Mar,Battle.net,0,1
3,battlenet_17459,"{""title"":""Diablo® III""}",2012 May,Battle.net,0,1
4,battlenet_21297,"{""title"":""StarCraft® Remastered""}",2017 Aug,Battle.net,0,1
...,...,...,...,...,...,...
652,xboxone_964706972,"{""title"":""No Man's Sky""}",2016 Aug,Xbox Gamepass,0,1
653,xboxone_968775939,"{""title"":""Everspace""}",2017 May,Xbox Gamepass,0,1
654,xboxone_974586226,"{""title"":""We Happy Few (Game Preview)""}",2018 Aug,Xbox Gamepass,0,1
655,xboxone_979032440,"{""title"":""Supermarket Shriek""}",2019 Jun,Xbox Gamepass,0,1
